In [8]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [2]:
# Create Vector DB
def get_vector_db(df):

    # Uncomment for persistent client
    chroma_client = chromadb.PersistentClient()

    EMBEDDING_MODEL = "text-embedding-ada-002"
    # change this to biotech specialised model later
    embedding_function = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name=EMBEDDING_MODEL)
    scrnatools_description_collection = chroma_client.create_collection(name='scRNA_Tools', embedding_function=embedding_function)

    # Add the content vectors
    scrnatools_description_collection.add(
        documents = list(df['extented_desc']),
        metadatas = df.drop(['extented_desc'], axis = 1).to_dict(orient='records'),
        ids = list(df.Name)
    )

    scrnatools_description_collection.add(
        documents = list(df['extented_desc_readme_trim']),
        metadatas = df.drop(['extented_desc_readme_trim'], axis = 1).to_dict(orient='records'),
        ids = list(df.Name))
    
    return scrnatools_description_collection

In [4]:
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

In [5]:
df_eariched = pd.read_csv('dataframes/tool-table-with-readmes.csv')

In [ ]:
scrnatools_description_collection = get_vector_db(df_eariched)

In [14]:
scrnatools_description_collection

Collection(name=scRNA_Tools)

In [15]:
# Query DB
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances'])
    result = pd.DataFrame({
                'id':results['ids'][0],
                'score':results['distances'][0],
                'content': dataframe[dataframe.Name.isin(results['ids'][0])]['extented_desc'],
                'platform': dataframe[dataframe.Name.isin(results['ids'][0])]['Platform'],
                })

    return result

In [16]:
query_collection(scrnatools_description_collection, 'python', 3, df_eariched)

,id,score,content,platform
964,scAMACE_py,0.382842,Platform: Python\n Description: Python impleme...,Python
1465,uniPort,0.385923,Platform: Python\n Description: A unified appr...,Python
1575,SpatialScope,0.397222,Platform: Python\n Description: A unified comp...,Python
